In [1]:
import csv
import logging
import multiprocessing
import os
import subprocess
from multiprocessing import Pool
from subprocess import check_output
import logging
import multiprocessing
import random
from itertools import product, repeat
import itertools
import PIL.Image
import numpy as np
import torchvision
from PIL import Image
from bird_or_bicycle import metadata
from bird_or_bicycle.metadata import NUM_IMAGES_PER_CLASS
from tqdm import tqdm
import tensorflow_addons as tfa
import torchvision.transforms.functional
from torchvision.io import read_image
import torch
import torch.nn as nn
import torchvision.transforms as T
import matplotlib.pyplot as plt
import torchvision.models as models
from unrestricted_advex.eval_kit import logits_to_preds
from bird_or_bicycle.iterator import *
from unrestricted_advex.eval_kit import *
from unrestricted_advex.eval_kit import _validate_logits

BICYCLE_IDX = 0
BIRD_IDX = 1

/nethome/mhull32/miniconda3/envs/unr_adv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-08-17 18:38:52.908675: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
def show(imgs):
    fix, axs = plt.subplots(ncols=len(imgs), squeeze=False)
    for i, img in enumerate(imgs):
        img = T.ToPILImage()(img.to('cpu')) 
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])


In [3]:
# bike_path  = '/nvmescratch/mhull32/unrestricted-adversarial-examples/data_generation/extras/bicycle/blender_0_black.jpg'
# bird_path = '/nethome/mhull32/datasets/bird_or_bicycle/0.0.4/train/bird/083ba62f93396358.jpg'
# bike = read_image(bike_path)
# bird = read_image(bird_path)
# show([bike, bird])

In [4]:
checkpoint = torch.load('/nvmescratch/mhull32/unrestricted-adversarial-examples/model_zoo/undefended_pytorch_resnet.pth.tar')
model = getattr(models, checkpoint['arch'])(num_classes=2)
model = nn.Sequential(nn.BatchNorm2d(num_features=3, affine=False), model)
model = torch.nn.DataParallel(model).cuda()
model.load_state_dict(checkpoint['state_dict'])

def wrapped_model(x_np):
    x_np = x_np.transpose((0, 3, 1, 2))  # from NHWC to NCHW
    x_t = torch.from_numpy(x_np).cuda()
    model.eval()
    with torch.no_grad():
        return model(x_t).cpu().numpy()

In [5]:
data_root='/nvmescratch/mhull32/unrestricted-adversarial-examples/data_generation'

In [12]:
def _get_iterator(batch_size=4, shuffle=True,
                 verify_dataset=True):
  """ Create a backend-agnostic iterator for the dataset.
  Images are formatted in channels-last in the Tensorflow style
  :param split: One of ['train', 'test', 'extras']
  :param batch_size: The number of images and labels in each batch
  :param shuffle: Whether or not to shuffle
  :return:  An iterable that returns (batched_images, batched_labels, batched_image_ids)
  """
  data_dir = bird_or_bicycle.get_dataset(split='augmented', data_root = data_root, verify=True)

  image_preprocessing = torchvision.transforms.Compose([
    torchvision.transforms.Resize(224),
    torchvision.transforms.ToTensor(),
    lambda x: torch.einsum('chw->hwc', [x]),
  ])

  train_dataset = ImageFolderWithFilenames(
    data_dir, transform=image_preprocessing
  )

  data_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=shuffle)

  assert train_dataset.class_to_idx['bicycle'] == BICYCLE_IDX
  assert train_dataset.class_to_idx['bird'] == BIRD_IDX

  dataset_iter = ((x_batch.numpy(), y_batch.numpy(), image_ids)
                  for (x_batch, y_batch, image_ids) in iter(data_loader))

  return dataset_iter

dataset_iter = _get_iterator(batch_size=1)

for i_batch, (x_np, y_np, image_ids) in enumerate(tqdm(dataset_iter)):
   # print(i_batch)
    print(image_ids)

Verification of dataset successful. Dataset is correctly prepared.


FileNotFoundError: Found no valid file for the classes test. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp

In [ ]:
for i_batch, (x_np, y_np, image_ids) in enumerate(tqdm(dataset_iter)):
    assert x_np.shape[-1] == 3 or x_np.shape[-1] == 1, "Data was {}, should be NHWC".format(
      x_np.shape)
    # x_adv = attack_fn(model, x_np, y_np)s
    x_adv = x_np
    logits = wrapped_model(x_adv)
    correct = np.equal(logits_to_preds(logits), y_np).astype(np.float32)
    _validate_logits(logits, batch_size=len(x_np))